In [1]:
import pandas as pd

In [4]:
file_path = r'C:\Users\이현승\핀인사이트 시정 분석 프로젝트\data_extracted_keywords_top10.csv'
data = pd.read_csv(file_path)
data

,extracted_keywords
0,"['국내온라인판매', '온라인판매강화', '판매강화제품군', '강화제품군확대', '..."
1,"['삼성태블릿신제품', '태블릿신제품갤럭시', '신제품갤럭시탭', '시리즈’출시',..."
2,"['美상호관세', '상호관세폭탄', '관세폭탄숨통', '숨통튼韓', '韓반도체불확실..."
3,"['트럼프관세쇼크', '쇼크에韓', '韓전자업계', '전자업계초긴장', '업계초긴장..."
4,"['베트남46삼성', '삼성·LG전자', 'LG전자해외공장', '외공장도고민', '..."
5,"['삼성전자TV넷플릭스', '기술적용', '적용삼성전자TV', '선명한화질', '화..."
6,"['TV로넷플릭스', '선명한이유', '이유콘텐츠', '적용삼성전자TV', 'TV와..."
7,"['이재용삼성전자', '삼성전자회장광폭', '회장광폭행보', '광폭행보중국', '최..."
8,"['삼성태블릿신제품', '태블릿신제품갤럭시', '신제품갤럭시탭', '시리즈오늘국내'..."
9,"['삼성전자갤럭시탭', '시리즈출시', '69만', '삼성전자가태블릿', '태블릿신..."


In [ ]:
import pandas as pd
import ast  
import re

def safe_literal_eval(s):
    """CSV의 문자열 형식 리스트를 안전하게 변환"""
    try:
        s = str(s).strip()              # 외부 대괄호 및 공백 제거       
        s = re.sub(r"^\[|\]$", "", s)   # 내부 작은따옴표 -> 큰따옴표 통일      
        s = re.sub(r"'", '"', s)        # 빈 문자열 처리
        if not s: return []
        return ast.literal_eval(f'[{s}]')   # 문자열을 Python 객체로 변환.
    except:
        return []

def extract_aspects(keywords):
    """복합 명사를 분리하여 Aspect 추출"""
    aspects = []
    for keyword in keywords:
        # 2~4글자 단위로 분할 (예: '국내온라인판매' -> ['국내', '온라인', '판매'])
        for length in [4, 3, 2]:  # 긴 단어 우선 추출
            for i in range(len(keyword) - length + 1):
                aspect_candidate = keyword[i:i+length]
                # 의미 있는 단어 필터링
                if any(sub in aspect_candidate for sub in ['전자', '판매', '관세', '제품']):
                    aspects.append(aspect_candidate)
    return list(set(aspects))  # 중복 제거

# CSV 파일 로드 및 전처리
data = pd.read_csv("data_extracted_keywords_top10.csv")
data["extracted_keywords"] = data["extracted_keywords"].apply(safe_literal_eval)

# Aspect 추출 실행
data["aspects"] = data["extracted_keywords"].apply(extract_aspects)

# 결과 저장
data.to_csv("aspect_results.csv", index=False)
print("Aspect 추출 완료! 결과 파일: aspect_results.csv")



Aspect 추출 완료! 결과 파일: aspect_results.csv


In [24]:
import pandas as pd
from transformers import pipeline

# CSV 파일 로드 (Aspect 추출 완료된 파일)
data = pd.read_csv("aspect_results.csv")  # 이전 단계에서 저장한 파일

# Hugging Face의 한국어 감성 분석 모델 초기화
sentiment_analyzer = pipeline(
    "text-classification",
    model="snunlp/KR-FinBert-SC",  # 한국어 감성 분석 모델
    tokenizer="snunlp/KR-FinBert-SC"
)

def perform_absa(aspects):
    """
    각 Aspect에 대해 감성 분석 수행.
    :param aspects: Aspect 리스트 (예: ['배터리', '디스플레이'])
    :return: [(Aspect, Sentiment)] 형태의 리스트
    """
    results = []
    for aspect in aspects:
        try:
            sentiment = sentiment_analyzer(aspect)[0]['label']  # 감성 분석 수행
            results.append((aspect, sentiment))
        except Exception as e:
            print(f"Error analyzing sentiment for aspect '{aspect}': {e}")
            continue
    return results

# ABSA 실행: 각 행의 'aspects' 열에 대해 감성 분석 수행
data["absa_results"] = data["aspects"].apply(
    lambda x: perform_absa(eval(x)) if isinstance(x, str) else []
)

# 결과 저장
output_file = "final_absa_results.csv"
data.to_csv(output_file, index=False)
print(f"ABSA 결과가 '{output_file}'에 저장되었습니다.")


Device set to use cpu


ABSA 결과가 'final_absa_results.csv'에 저장되었습니다.


In [36]:
file_path = "final_absa_results.csv"
df = pd.read_csv(file_path)

# 출력 옵션 설정 (생략 없이 표시)
pd.set_option('display.max_colwidth', None)

# 'absa_results' 컬럼만 선택하여 왼쪽 정렬 스타일 적용
styled_df = df[['absa_results']].style.set_properties(**{'text-align': 'left'})

styled_df



,absa_results
0,"[('국내전자', 'neutral'), ('매제품군', 'neutral'), ('제품및시', 'neutral'), ('제품및', 'neutral'), ('폼판매', 'neutral'), ('매제품', 'neutral'), ('TV제품', 'neutral'), ('전자와', 'neutral'), ('판매강', 'neutral'), ('판매에', 'neutral'), ('와전자', 'neutral'), ('제품도', 'neutral'), ('인판매처', 'neutral'), ('삼성전자', 'positive'), ('G전자', 'neutral'), ('제품은', 'neutral'), ('전자상거', 'positive'), ('강화제품', 'neutral'), ('판매전략', 'neutral'), ('제품을비', 'neutral'), ('전자랜', 'neutral'), ('해판매', 'neutral'), ('해판매제', 'neutral'), ('화제품군', 'neutral'), ('화제품', 'neutral'), ('제품으', 'neutral'), ('전제품을', 'neutral'), ('제품은T', 'neutral'), ('트와전자', 'neutral'), ('판매제', 'neutral'), ('라인판매', 'positive'), ('성전자와', 'negative'), ('판매에집', 'neutral'), ('내전자', 'neutral'), ('제품을', 'neutral'), ('전제품도', 'positive'), ('을판매', 'neutral'), ('쟁제품', 'neutral'), ('판매강화', 'neutral'), ('판매실적', 'neutral'), ('제품도취', 'neutral'), ('판매', 'neutral'), ('할전자', 'neutral'), ('랫폼판매', 'neutral'), ('제품군', 'neutral'), ('전제품', 'neutral'), ('할전자상', 'neutral'), ('인판매', 'neutral'), ('제품으로', 'neutral'), ('인판매강', 'neutral'), ('판매처', 'neutral'), ('판매제품', 'neutral'), ('판매실', 'neutral'), ('제품군확', 'neutral'), ('제품군을', 'neutral'), ('판매와관', 'neutral'), ('올해판매', 'neutral'), ('가전제품', 'neutral'), ('내전자상', 'positive'), ('경쟁제품', 'neutral'), ('전자와L', 'neutral'), ('LG전자', 'positive'), ('판매전', 'neutral'), ('성전자', 'negative'), ('전자', 'neutral'), ('제품', 'neutral'), ('인판매전', 'neutral'), ('전자랜드', 'neutral'), ('품을판매', 'neutral'), ('V제품', 'neutral'), ('전자상', 'positive'), ('제품을판', 'neutral'), ('무할전자', 'neutral'), ('판매와', 'neutral')]"
1,"[('신제품', 'positive'), ('별도판매', 'neutral'), ('제품매', 'neutral'), ('와전자', 'neutral'), ('성전자가', 'negative'), ('삼성전자', 'positive'), ('신제품갤', 'positive'), ('제품갤', 'neutral'), ('전자가중', 'neutral'), ('도판매', 'neutral'), ('전자제', 'neutral'), ('어와전자', 'neutral'), ('판매', 'neutral'), ('릿신제품', 'neutral'), ('제품매장', 'neutral'), ('자제품매', 'neutral'), ('성전자', 'negative'), ('자제품', 'neutral'), ('전자', 'neutral'), ('제품', 'neutral'), ('전자가', 'neutral'), ('제품갤럭', 'neutral'), ('전자제품', 'positive')]"
2,"[('편관세', 'neutral'), ('호관세미', 'neutral'), ('목별관세', 'neutral'), ('관세에각', 'negative'), ('관세와', 'neutral'), ('관세이', 'neutral'), ('관세를', 'neutral'), ('비관세', 'neutral'), ('별관세율', 'neutral'), ('관세제외', 'neutral'), ('별관세', 'neutral'), ('관세가부', 'neutral'), ('삼성전자', 'positive'), ('관세문제', 'neutral'), ('기본관세', 'neutral'), ('관세발표', 'neutral'), ('호관세', 'positive'), ('관세율', 'neutral'), ('관세미적', 'negative'), ('관세를추', 'negative'), ('관세폭', 'negative'), ('국산제품', 'neutral'), ('관세가', 'neutral'), ('관세제', 'neutral'), ('보편관세', 'neutral'), ('호관세부', 'neutral'), ('편관세체', 'neutral'), ('산제품', 'neutral'), ('성전자와', 'negative'), ('관세폭탄', 'negative'), ('관세체계', 'neutral'), ('체에관세', 'neutral'), ('본관세', 'neutral'), ('전자와S', 'neutral'), ('관세를계', 'neutral'), ('관세를적', 'negative'), ('관세율은', 'neutral'), ('관세', 'neutral'), ('관세적', 'negative'), ('호관세제', 'neutral'), ('관세미', 'neutral'), ('관세에', 'negative'), ('호관세이', 'neutral'), ('관세적용', 'negative'), ('수에관세', 'neutral'), ('호관세폭', 'positive'), ('업별관세', 'neutral'), ('상호관세', 'neutral'), ('비관세장', 'neutral'), ('에관세', 'neutral'), ('관세체', 'neutral'), ('관세부과', 'negative'), ('관세를부', 'negative'), ('개별관세', 'neutral'), ('관세장벽', 'neutral'), ('성전자', 'negative'), ('관세부', 'neutral'), ('호관세발', 'neutral'), ('호관세적', 'positive'), ('관세발', 'negative'), ('전자', 'neutral'), ('제품', 'neutral'), ('관세와보', 'neutral'), ('종비관세', 'neutral'), ('관세이후', 'neutral'), ('가별관세', 'neutral'), ('관세문', 'neutral'), ('전자와', 'neutral'), ('관세장', 'neutral')]"
3,"[('G전자', 'neutral'), ('전자의경', 'neutral'), ('관세를', 'neutral'), ('관세를발', 'negative'), ('전자기업', 'neutral'), ('관세율', 'neutral'), ('럼프관세', 'neutral'), ('관세영향', 'neutral'), ('전자산업', 'neutral'), ('전자는', 'neutral'), ('체관세', 'neutral'), ('G전자의', 'positive'), ('제품군', 'neutral'), ('전자의', 'neutral'), ('전자는전', 'neutral'), ('전자00', 'neutral'), ('관세쇼크', 'negative'), ('상호관세', 'neutral'), ('의전자', 'neutral'), ('전자', 'neutral'), ('한관세', 'neutral'), ('전자0', 'neutral'), ('전자와', 'neutral'), ('韓전자', 'neutral'), ('프관세', 'neutral'), ('전략제품', 'neutra